In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
ratings = pd.read_csv('data/ratings (1).csv')

In [ ]:
courses = pd.read_csv('data/courses.csv')

In [ ]:
courses.head()

In [ ]:
ratings.head()

In [ ]:
n_ratings = len(ratings)
n_courses = ratings['courseId'].nunique()
n_users = ratings['userId'].nunique()

print(f"Number of ratings: {n_ratings}")
print(f"Number of unique courses: {n_courses}")
print(f"Number of unique users: {n_users}")
print(f"Average number of ratings per user: {round(n_ratings/n_users, 2)}")
print(f"Average number of ratings per course: {round(n_ratings/n_courses, 2)}")

In [ ]:
sns.countplot(x='rating', data=ratings)
plt.title("Distribution of course ratings", fontsize=14)
plt.show()

In [ ]:
print(f"Mean global rating: {round(ratings['rating'].mean(),2)}.")

In [ ]:
mean_ratings = ratings.groupby('userId')['rating'].mean()
print(f"Mean rating per user: {round(mean_ratings.mean(),2)}.")

In [ ]:
ratings['courseId'].value_counts()

In [ ]:
course_ratings = ratings.merge(courses, on='courseId')
# course_ratings
course_ratings['Course'].value_counts()[0:10]

In [ ]:
mean_ratings = ratings.groupby('courseId')[['rating']].mean()
lowest_rated = mean_ratings['rating'].idxmin()
# mean_ratings
courses[courses['courseId'] == lowest_rated]

In [ ]:
highest_rated = mean_ratings['rating'].idxmax()

courses[courses['courseId'] == highest_rated]

In [ ]:
ratings[ratings['courseId']==highest_rated]

In [ ]:
course_stats = ratings.groupby('courseId')['rating'].agg(['count', 'mean'])
course_stats.head()

In [ ]:
C = course_stats['count'].mean()
m = course_stats['mean'].mean()

print(f"Average number of ratings for a given course: {C:.2f}")
print(f"Average rating for a given course: {m:.2f}")

def bayesian_avg(ratings):
    bayesian_avg = (C*m + ratings.sum())/(C+ratings.count())
    return round(bayesian_avg, 3)

In [ ]:
# these are the two ratings for highest rated course
highest_rated = pd.Series([5,5])
bayesian_avg(highest_rated)

In [ ]:
bayesian_avg_ratings = ratings.groupby('courseId')['rating'].agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['courseId', 'bayesian_avg']
course_stats = course_stats.merge(bayesian_avg_ratings, on='courseId')

In [ ]:
course_stats = course_stats.merge(courses[['courseId', 'Course']])
# course_stats.sort_values()

In [ ]:
course_stats.sort_values(by='bayesian_avg')

In [ ]:
# top 5 highest rated courses
course_stats.sort_values(by='bayesian_avg', ascending=False).head()

In [ ]:
# top 5 lowest rated courses
course_stats.sort_values(by='bayesian_avg', ascending=True).head()

In [ ]:
courses['Related_Interests'] = courses['Related_Interests'].apply(lambda x: x.split('|'))
courses.head()

In [ ]:
from collections import Counter

interest_frequency = Counter(i for interests in courses['Related_Interests'] for i in interests)

print(f"There are {len(interest_frequency)} Related interests.")

interest_frequency

In [ ]:
print("The 5 most common interests: \n", interest_frequency.most_common(5))

In [ ]:
interest_frequency_df = pd.DataFrame([interest_frequency]).T.reset_index()
interest_frequency_df.columns= ['Related_Interests', 'count']

sns.barplot(x='Related_Interests', y='count', data=interest_frequency_df.sort_values(by='count', ascending=False).head())
plt.title("Distribution of Related Interests", fontsize=14)
plt.xticks(rotation=90)

In [ ]:
from scipy.sparse import csr_matrix

def create_X(df):
    """
    Generates a sparse matrix from ratings dataframe.

    Args:
        df: pandas dataframe containing 3 columns (userId, courseId, rating)

    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        course_mapper: dict that maps course id's to course indices
        course_inv_mapper: dict that maps course indices to course id's
    """
    M = df['userId'].nunique()
    N = df['courseId'].nunique()

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
    course_mapper = dict(zip(np.unique(df["courseId"]), list(range(N))))

    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
    course_inv_mapper = dict(zip(list(range(N)), np.unique(df["courseId"])))

    user_index = [user_mapper[i] for i in df['userId']]
    item_index = [course_mapper[i] for i in df['courseId']]

    X = csr_matrix((df["rating"], (user_index,item_index)), shape=(M,N))

    return X, user_mapper, course_mapper, user_inv_mapper, course_inv_mapper

X, user_mapper, course_mapper, user_inv_mapper, course_inv_mapper = create_X(ratings)

In [ ]:
X.shape

In [ ]:
n_total =X.shape[0]*X.shape[1]
n_ratings =X.nnz
sparsity = n_ratings/n_total
print(f"Matrix sparsity: {round(sparsity*100,2)}%")

In [ ]:
n_ratings_per_user = X.getnnz(axis=1)
len(n_ratings_per_user)

In [ ]:
n_ratings_per_course = X.getnnz(axis=0)
len(n_ratings_per_course)

In [ ]:
print(f"Most rated course has {n_ratings_per_course.max()} ratings.")
print(f"Least rated course has {n_ratings_per_course.min()} ratings.")

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(1,2,1)
sns.kdeplot(n_ratings_per_user, fill=True)
plt.xlim(0)
plt.title("Number of Ratings Per User", fontsize=14)
plt.xlabel("number of ratings per user")
plt.ylabel("density")
plt.subplot(1,2,2)
sns.kdeplot(n_ratings_per_course, fill=True)
plt.xlim(0)
plt.title("Number of Ratings Per course", fontsize=14)
plt.xlabel("number of ratings per course")
plt.ylabel("density")
plt.show()

In [ ]:
from sklearn.neighbors import NearestNeighbors

def find_similar_courses(course_id, X, course_mapper, course_inv_mapper, k, metric='cosine'):
    """
    Finds k-nearest neighbours for a given course id.

    Args:
        course_id: id of the course of interest
        X: user-item utility matrix
        k: number of similar courses to retrieve
        metric: distance metric for kNN calculations

    Output: returns list of k similar course ID's
    """
    X = X.T
    neighbour_ids = []

    course_ind = course_mapper[course_id]
    course_vec = X[course_ind]
    if isinstance(course_vec, (np.ndarray)):
        course_vec = course_vec.reshape(1,-1)
    # use k+1 since kNN output includes the courseId of interest
    kNN = NearestNeighbors(n_neighbors=k+1, algorithm="brute", metric=metric)
    kNN.fit(X)
    neighbour = kNN.kneighbors(course_vec, return_distance=False)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(course_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

In [ ]:
similar_courses = find_similar_courses(1, X, course_mapper, course_inv_mapper, k=10)
similar_courses

In [ ]:
# using cosine metrics
course_titles = dict(zip(courses['courseId'], courses['Course']))
# course_titles
course_id = 1

similar_courses = find_similar_courses(course_id, X, course_mapper, course_inv_mapper, metric='cosine', k=10)
course_title = course_titles[course_id]

print(f"Because you selected {course_title}:")
for i in similar_courses:
  print(course_titles[i])

In [ ]:
# using euclidean, you can replace it with manhattan for another view
course_titles = dict(zip(courses['courseId'], courses['Course']))
# course_titles
course_id = 1

similar_courses = find_similar_courses(course_id, X, course_mapper, course_inv_mapper, metric='euclidean', k=10)
course_title = course_titles[course_id]

print(f"Because you selected {course_title}:")
for i in similar_courses:
  print(course_titles[i])

In [ ]:
interests = set(i for I in courses['Related_Interests'] for i in I)
# interests
for i in interests:
    courses[i] = courses.Related_Interests.transform(lambda x: int(i in x))

course_interests = courses.drop(columns=['courseId', 'Course','Related_Interests', 'Faculty', 'Level'])

In [ ]:
course_interests.head()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(course_interests, course_interests)
print(f"Dimensions of our interests cosine similarity matrix: {cosine_sim.shape}")

In [ ]:
from fuzzywuzzy import process
import Levenshtein

def course_finder(title):
    all_titles = courses['Course'].tolist()
    closest_match = process.extractOne(title, all_titles)
    return closest_match[0]

In [ ]:
title = course_finder('finance')
title

In [ ]:
course_idx = dict(zip(courses['Course'], list(courses.index)))
idx = course_idx[title]
print(f"course index for Account: {idx}")

In [ ]:
n_recommendations=10
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
sim_scores=sim_scores[1:n_recommendations+1]
sim_scores


In [ ]:
similar_courses= [i[0] for i in sim_scores]
similar_courses

In [ ]:
print(f"Because you selected {title}:")
courses['Course'].iloc[similar_courses]

In [ ]:
def get_content_based_recommendations(title_string, n_recommendations=10):
    title = course_finder(title_string)
    idx = course_idx[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:(n_recommendations+1)]
    similar_courses = [i[0] for i in sim_scores]
    print(f"Because you selected {title}:")
    print(courses['Course'].iloc[similar_courses])

In [ ]:
# provide the course keywords and how many recommendations you would like to
# generate
get_content_based_recommendations('it', 5)

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=20, n_iter=10)
Q = svd.fit_transform(X.T)
Q.shape

In [ ]:
course_id = 1
similar_courses = find_similar_courses(course_id, Q.T, course_mapper, course_inv_mapper, metric='cosine', k=10)
course_title = course_titles[course_id]

print(f"Because you selected {course_title} similar courses are:")
for i in similar_courses:
    print(course_titles[i])